In [138]:
import pandas as pd
import numpy as np
import os

In [139]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

In [140]:
files = os.listdir("/kaggle/input/seed-power-quality-disturbance-dataset/XPQRS")
files = [i for i in files if i[-3:]== "csv"]

In [141]:
files

['Pure_Sinusoidal.csv',
 'Swell_with_Harmonics.csv',
 'Transient.csv',
 'Harmonics.csv',
 'Harmonics_with_Sag.csv',
 'Swell.csv',
 'Swell_with_Oscillatory_Transient.csv',
 'Sag.csv',
 'Interruption.csv',
 'Flicker.csv',
 'Harmonics_with_Swell.csv',
 'Flicker_with_Swell.csv',
 'Notch.csv',
 'Flicker_with_Sag.csv',
 'Sag_with_Harmonics.csv',
 'Oscillatory_Transient.csv',
 'Sag_with_Oscillatory_Transient.csv']

In [142]:
def compute_fft_features(signal, sampling_rate=1.0, n_features=10):
    """
    Compute top n_features magnitudes and their corresponding frequencies.
    """
    n = len(signal)
    fft_vals = np.fft.fft(signal)
    fft_freqs = np.fft.fftfreq(n, d=1/sampling_rate)

    # Keep only the positive half (real signal)
    half_n = n 
    fft_magnitudes = np.abs(fft_vals[:half_n])
    fft_freqs = fft_freqs[:half_n]
    # Select top `n_features` based on magnitude

    top_indices = np.argsort(fft_magnitudes)[-n_features:]
    top_magnitudes = fft_magnitudes[top_indices]
    top_frequencies = fft_freqs[top_indices]

    return top_magnitudes, top_frequencies


In [143]:
data = []
labels = []
for idx,i in enumerate(files):
    path = "/kaggle/input/seed-power-quality-disturbance-dataset/XPQRS"
    csv = os.path.join(path,i)
    df = pd.read_csv(csv)
    for _,row in df.iterrows():
        seq = row.values.astype(np.float32)
        
        mag,freq = compute_fft_features(seq,sampling_rate = 5000,n_features = 100)
        mag,freq = mag.astype(float),freq.astype(float)
        features = np.array([seq,mag,freq])
        data.append(features)
        labels.append(idx)

In [144]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Reshape to 2D array as required by sklearn
labels = np.array(labels).reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)  # use sparse=True for sparse matrix
onehot_labels = encoder.fit_transform(labels)



/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [145]:
len(data)

16983

In [146]:
class dataload(Dataset):
    def __init__(self,X_train,Y_train):
        super().__init__()
        self.x = X_train
        self.y = Y_train
    def __len__(self):
        return len(self.x)
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
        

In [147]:
dataset = dataload(data,onehot_labels)


In [148]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # shape: (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim=3, model_dim=64, num_heads=4, num_layers=2, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.input_projection = nn.Linear(input_dim, model_dim)
        self.positional_encoding = PositionalEncoding(model_dim)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.output_layer = nn.Linear(model_dim, 17)  # output_dim = 1 for regression

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        x = self.input_projection(x)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)               
        return self.output_layer(x)

In [149]:
model = TimeSeriesTransformer().to("cuda")
optimizer = Adam(model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()


In [150]:
device = torch.device("cuda")

In [151]:
from sklearn.model_selection import train_test_split

train_data,test_data = train_test_split(dataset,test_size = 0.1)

In [179]:
train_loader = DataLoader(train_data,batch_size = 256,shuffle = True)
test_loader = DataLoader(test_data,batch_size = 256,shuffle = True)




for epoch in range(100):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)

        targets = targets.to(device)
        optimizer.zero_grad()
        inputs = inputs.permute(0,2,1)
        inputs = inputs.float()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1},Traning Loss: {running_loss / len(train_loader):.4f}")
    running_loss = 0.0
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        inputs  = inputs.permute(0,2,1)
        inputs = inputs.float()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Validation Loss: {running_loss / len(test_loader):.4f}")


Epoch 1,Traning Loss: 0.2615
Epoch 1, Validation Loss: 0.4025
Epoch 2,Traning Loss: 0.2775
Epoch 2, Validation Loss: 0.4040
Epoch 3,Traning Loss: 0.2917
Epoch 3, Validation Loss: 0.4199
Epoch 4,Traning Loss: 0.2761
Epoch 4, Validation Loss: 0.4033
Epoch 5,Traning Loss: 0.2718
Epoch 5, Validation Loss: 0.3911
Epoch 6,Traning Loss: 0.2815
Epoch 6, Validation Loss: 0.4129
Epoch 7,Traning Loss: 0.2696
Epoch 7, Validation Loss: 0.4017
Epoch 8,Traning Loss: 0.2691
Epoch 8, Validation Loss: 0.4233
Epoch 9,Traning Loss: 0.2782
Epoch 9, Validation Loss: 0.3910
Epoch 10,Traning Loss: 0.2675
Epoch 10, Validation Loss: 0.3940
Epoch 11,Traning Loss: 0.2655
Epoch 11, Validation Loss: 0.4088
Epoch 12,Traning Loss: 0.2780
Epoch 12, Validation Loss: 0.4271
Epoch 13,Traning Loss: 0.2728
Epoch 13, Validation Loss: 0.3933
Epoch 14,Traning Loss: 0.2805
Epoch 14, Validation Loss: 0.4013
Epoch 15,Traning Loss: 0.2601
Epoch 15, Validation Loss: 0.3982
Epoch 16,Traning Loss: 0.2746
Epoch 16, Validation Loss: 0

In [182]:
# Save the model's state_dict
torch.save(model.state_dict(), 'model_weights2.pth')


In [181]:
import torch
import numpy as np

# Assuming: model, test_loader, and device are already defined
model.eval()

n_classes = 17
class_correct = [0 for _ in range(n_classes)]
class_total = [0 for _ in range(n_classes)]

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.permute(0,2,1)
        inputs = inputs.float()

        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        labels = torch.argmax(labels,dim = 1)
        for i in range(len(labels)):
            label = labels[i].item()
            pred = preds[i].item()
            if label == pred:
                class_correct[label] += 1
            class_total[label] += 1

# Print class-wise accuracy
for i in range(n_classes):
    accuracy = 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
    print(f"Class {i}: Accuracy = {accuracy:.2f}%")
print(class_total)

Class 0: Accuracy = 100.00%
Class 1: Accuracy = 59.22%
Class 2: Accuracy = 96.40%
Class 3: Accuracy = 100.00%
Class 4: Accuracy = 76.84%
Class 5: Accuracy = 100.00%
Class 6: Accuracy = 77.78%
Class 7: Accuracy = 97.70%
Class 8: Accuracy = 81.82%
Class 9: Accuracy = 100.00%
Class 10: Accuracy = 91.21%
Class 11: Accuracy = 78.00%
Class 12: Accuracy = 93.81%
Class 13: Accuracy = 67.95%
Class 14: Accuracy = 44.86%
Class 15: Accuracy = 86.14%
Class 16: Accuracy = 84.54%
[106, 103, 111, 111, 95, 114, 99, 87, 99, 103, 91, 100, 97, 78, 107, 101, 97]


In [ ]:
Class 0: Accuracy = 100.00%
Class 1: Accuracy = 12.62%
Class 2: Accuracy = 95.50%
Class 3: Accuracy = 99.10%
Class 4: Accuracy = 24.21%
Class 5: Accuracy = 99.12%
Class 6: Accuracy = 32.32%
Class 7: Accuracy = 80.46%
Class 8: Accuracy = 98.99%
Class 9: Accuracy = 100.00%
Class 10: Accuracy = 57.14%
Class 11: Accuracy = 55.00%
Class 12: Accuracy = 81.44%
Class 13: Accuracy = 37.18%
Class 14: Accuracy = 23.36%
Class 15: Accuracy = 83.17%
Class 16: Accuracy = 31.96%


In [ ]:
['Pure_Sinusoidal.csv',
 'Swell_with_Harmonics.csv',
 'Transient.csv',
 'Harmonics.csv',
 'Harmonics_with_Sag.csv',
 'Swell.csv',
 'Swell_with_Oscillatory_Transient.csv',
 'Sag.csv',
 'Interruption.csv',
 'Flicker.csv',
 'Harmonics_with_Swell.csv',
 'Flicker_with_Swell.csv',
 'Notch.csv',
 'Flicker_with_Sag.csv',
 'Sag_with_Harmonics.csv',
 'Oscillatory_Transient.csv',
 'Sag_with_Oscillatory_Transient.csv']
